In [2]:
### Optimization of RBM ###
from __future__ import division, print_function, absolute_import
import os
import sys
sys.path.append('/glade/work/dblaskey/RBM_opt_code/src')
import MOASMO
import Gen_Val_Inputs
import convert_rbm_to_nc
from convert_rbm_to_nc import read_config
import numpy as np
import matplotlib.pyplot as plt
import util
import _pickle as cPickle
import pandas as pd
import sampling
import xarray as xr
import seaborn as sn
import GLP
import gp
import NSGA2
import pickle
import smt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import random
import re
from sklearn.metrics import mean_squared_error
import cmocean
import hydroeval as he
import glob
import subprocess
import multiprocessing
import math
import warnings
from scipy.ndimage import gaussian_filter1d

In [3]:
#### Begin Code #####
df_sites = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/Opt_Basins.csv', index_col=0)
df_sites = df_sites[df_sites['type']=="Opt"]
outlets = np.unique(df_sites.outlet_comid.values)

# Read in Observed Data
temp_data = pd.read_csv('/glade/scratch/dblaskey/RBM/temperature_gages.csv', index_col=0)
df = pd.merge(temp_data, df_sites, on="site_no")

psets_df = pd.read_csv('/glade/u/home/dblaskey/RBM/Optimization/Opt_runs_0.csv')
psets_df.rename(columns={ psets_df.columns[0]: "Name" }, inplace = True)

years = range(2013,2018)
start_date = '2013-01-01'
end_date = '2017-12-31'
time_series = pd.date_range(start_date, end_date)

for folder in np.unique(psets_df.Name.values):
    # Build Output Folders
    path = '/glade/scratch/dblaskey/RBM/Regression/Opt/test/%s/'%folder
    if os.path.exists(path) == False:           
        print("Creating ", folder)
        os.mkdir(path)

In [4]:
def cal_regression(folder):
    for outlet in outlets:
        print(folder, outlet)
        
        var_list = psets_df[psets_df['Name'] == folder]

        alpha = var_list.alpha.values[0]
        beta = var_list.beta.values[0]
        mu = var_list.mu.values[0]
        gamma = var_list.gamma.values[0]

        Ordered_reaches_final = pd.read_csv("/glade/scratch/dblaskey/RBM/Input/%s_network.csv" % outlet)

        results = []

        for i, comid in enumerate(df_sites[df_sites['outlet_comid'] == outlet]['COMID'].values):
            cell = Ordered_reaches_final[Ordered_reaches_final['hru'] == comid].node.values[0]
            site_no = df_sites[df_sites['outlet_comid'] == outlet].index.values[i]

            Water = []
            for year in years:
                test = pd.read_csv("/glade/scratch/dblaskey/RBM/RBM_Input/%s_energy_%s" % (outlet, year), sep=" ", header=None, names=["cell", "Tair", "vp", "SW", "LW", "Density", "P", "Wind"])
                test_id = test[test["cell"] == cell]
                test_id.drop(test_id.tail(1).index, inplace=True)
                test_id['T_smooth'] = 0.1 * test_id['Tair'] + 0.9 * test_id['Tair'].shift()
                test_id['T_smooth'] = test_id['T_smooth'].fillna(test_id['Tair'])
                
                # Calculate air temperature
                def mosheni(air_temp, alpha, beta, mu, gamma):
                    return mu + (alpha - mu)/(1 + math.exp(gamma*(beta-air_temp)))

                
                Water.append(test_id.apply(lambda row : mosheni(row['T_smooth'], alpha, beta, mu, gamma), axis=1))

            Water = np.concatenate(Water)                        
            sim_df = pd.DataFrame(Water, index=time_series, columns=['sim'])
            sim_df.index.name = 'Date'

            # Filter to just one gage for observations
            temp_df = df[df['site_no'] == site_no][['X_00010_00003', 'Date']]
            temp_df = temp_df.rename(columns={"X_00010_00003": "obs"})
            temp_df = temp_df.set_index('Date')
            temp_df.index = pd.to_datetime(temp_df.index)

            # Combine simulation and observation datasets
            df_concat = pd.concat([sim_df, temp_df], axis=1)
            df_concat = df_concat.dropna()
            df_concat = df_concat.loc['2013-10-01':'2017-09-30']
            df_concat = df_concat[df_concat.index.month.isin([5, 6, 7, 8, 9])]
            df_concat = df_concat.reset_index()

            df_concat.to_csv('/glade/scratch/dblaskey/RBM/Regression/Opt/test/%s/%s.csv' % (folder, comid))

            result = {
                'RMSE': he.evaluator(he.rmse, df_concat.sim.values, df_concat.obs.values)[0],
                'COMID': np.int(comid),
                'Name': folder,
                'Outlet': outlet
            }

            results.append(result)

    return results

#### Preprocess RBM #### 
# set up the multiprocessing pool
pool = multiprocessing.Pool()

# run the function in parallel using the multiprocessing pool
results = pool.map(cal_regression, np.unique(psets_df.Name.values)[:2])

# close the multiprocessing pool
pool.close() 

pe_basin_0_0001pe_basin_0_0000  8100000581000005



/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

pe_basin_0_0001pe_basin_0_0000  8100043381000433



/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

pe_basin_0_0001pe_basin_0_0000  8102002181020021



...

/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

pe_basin_0_0000pe_basin_0_0001  8103091281030912



/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

pe_basin_0_0000 81035794
pe_basin_0_0001 81035794


/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [5]:
results

[[{'RMSE': 2.3479837543978666,
   'COMID': 81034272,
   'Name': 'pe_basin_0_0000',
   'Outlet': 81035794}],
 [{'RMSE': 3.3682938693362527,
   'COMID': 81034272,
   'Name': 'pe_basin_0_0001',
   'Outlet': 81035794}]]

In [6]:
# Flatten the list of lists into a single list
results = [item for sublist in results for item in sublist]

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results)

results_df

,RMSE,COMID,Name,Outlet
0,2.347984,81034272,pe_basin_0_0000,81035794
1,3.368294,81034272,pe_basin_0_0001,81035794
